In [36]:
import numpy as np
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from nibabel.testing import data_path
import nibabel as nib
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import time
import datetime
import json
from collections import defaultdict

In [37]:
import itertools
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from sklearn import preprocessing

In [38]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2023-09-16 22:46:14.423620: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30967 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:07:00.0, compute capability: 7.0


In [22]:
file_list = glob(os.path.join
                 ("/home/tordatom/Dati_Imaging/BraTs_19/MICCAI_BraTS_2019_Data_Training/HGG",
                  "*", ""))

In [9]:
 exam_path = [np.sort(glob(os.path.join(file_list[i], "*.nii.gz"))) for i in range(len(file_list))]

In [10]:
def load_example(file_list):
    
    exam_path = [np.sort(glob(os.path.join(file_list[i], "*.nii.gz"))) for i in range(len(file_list))]
    seg = np.zeros(np.append(len(file_list),nib.load(exam_path[0][0]).shape))
    channels = np.zeros(np.append(seg.shape,4))
    file_name = [i for i in file_list]
    
    for i in range(len(file_list)):
       
    
        seg[i] = nib.load(exam_path[i][1]).dataobj
    
        for j,k in enumerate(sorted(list(set(exam_path[i]) - set([exam_path[i][1]])))):
            
            a = nib.load(k)
            channels[i,:,:,:,j] = np.array(a.dataobj)
    

    return channels, seg, file_name

In [11]:
def normalize(t1):
    for i in range(len(t1[:,0,0,0,0])):
        for j in range(len(t1[0,0,0,0,:])):
            t1[i,:,:,:,j] = 2*t1[i,:,:,:,j]/np.max(t1[i,:,:,:,j])-1
    return t1



In [12]:
def Twod(channels):
    new = np.zeros((channels.shape[0]*channels.shape[3], channels.shape[1], channels.shape[2], channels.shape[4]))
    for i in range(channels.shape[0]):
        for j in range(channels.shape[3]):
            new[channels.shape[3]*i+j] = channels[i,:,:,j,:]
    return new

In [13]:
from sklearn.model_selection import train_test_split
train_to_test_ratio=0.9

In [14]:
channels1, seg1, file_name = load_example(file_list)

In [15]:
#channels1, seg1, file_name = load_example(file_list)
channels, seg = channels1[:,27:219, 27:219, 90:100, :], seg1[:,27:219, 27:219, 90:100]
channels = normalize(channels)
seg = seg.reshape(np.append(seg.shape,1))
seg[seg == 4] = 3
seg = keras.utils.to_categorical(seg)

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(np.array(channels),
                                               np.array(seg),train_size=train_to_test_ratio,
                                               shuffle=True, random_state=1234)

X_train, Y_train = Twod(X_train),  Twod(Y_train)
X_test, Y_test = Twod(X_test), Twod(Y_test)


In [76]:
for i in np.arange(0,X_train.shape[0], BATCH):
    
    file_name_train = 'Train_{0}'.format(i//BATCH)
    if i+BATCH>=X_train.shape[0]:
        np.savez(os.path.join('/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Data/DataTracin_train4' , file_name_train), 
                 X_train = X_train[i:], Y_train = Y_train[i:])
        
    np.savez(os.path.join('/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Data/DataTracin_train4' , file_name_train),
             X_train = X_train[i:i+BATCH], Y_train = Y_train[i:i+BATCH])

In [59]:
for i in np.arange(0,X_test.shape[0], BATCH):
    
    file_name_test = 'Test_{0}'.format(i//BATCH)
    if i+BATCH>=X_test.shape[0]:
        np.savez(os.path.join('/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Data/DataTest' , file_name_test), 
                 X_test = X_test[i:], Y_test = Y_test[i:])
        
    np.savez(os.path.join('/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Data/DataTest' , file_name_test),
             X_test = X_test[i:i+BATCH], Y_test = Y_test[i:i+BATCH])